# Table of Contents
1. [download/read csv file](#sec1)
2. [Convert the labeled data set tags into the new individual tag columns](#sec2)
3. [NLTK STUFF LMAO](#sec3)

In [3]:
#packages
%pip install numpy
%pip install pandas
import numpy as np
import pandas as pd


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\jessi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\jessi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


<a id = "sec1"></a>
## **Download/Read CSV file**

In [5]:
# read csv
data_path= "https://raw.githubusercontent.com/JL72005/PIT-UN-Project4/refs/heads/main/MODIFIED_dc_md_va_flash_floods_1996_pres.csv?token=GHSAT0AAAAAADCLZGCQLZ3HT2MKGULJWVAQ2B7HCLA"
df = pd.read_csv(data_path)

<a id = "sec2"></a>
## **Convert the labeled data set tags into the new individual tag columns**

In [6]:
#list of OG tags
['death','injury','evacuation','rescue','car_crash','home_damage','infrastructure_damage','school_damage','hospital_damage','road_closure','power_outage']

['death',
 'injury',
 'evacuation',
 'rescue',
 'car_crash',
 'home_damage',
 'infrastructure_damage',
 'school_damage',
 'hospital_damage',
 'road_closure',
 'power_outage']

In [7]:
#populate columns for OG tags
df['death'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'death' in x else 0)
df['injury'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'injury' in x else 0)
df['evacuation'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'evacuation' in x else 0)
df['rescue'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'rescue' in x else 0)
df['car_crash'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'car_crash' in x else 0)
df['home_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'home_damage' in x else 0)
df['infrastructure_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'infrastructure_damage' in x else 0)
df['school_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'school_damage' in x else 0)
df['hospital_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'hospital_damage' in x else 0)
df['road_closure'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'road_closure' in x else 0)
df['power_outage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'power_outage' in x else 0)
df['tree_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'tree_damage' in x else 0)
df['vehicle_loss'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'vehicle_loss' in x else 0)
df['agricultural_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'animal_loss' in x else 0)
df['campground_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'tree_damage' in x else 0)

#weather events
df['thunderstorm'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'thunderstorm' in x else 0)
df['hurricane'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'hurricane' in x else 0)
df['tornado'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'thunderstorm' in x else 0)
df['lightning'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'thunderstorm' in x else 0)
df['mudslide'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'mudslide' in x else 0)

In [8]:
#check columns
df[['death','injury','evacuation','rescue','car_crash','home_damage','infrastructure_damage','school_damage','hospital_damage','road_closure','power_outage','tree_damage']].head()

,death,injury,evacuation,rescue,car_crash,home_damage,infrastructure_damage,school_damage,hospital_damage,road_closure,power_outage,tree_damage
0,0,0,0,0,0,1,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1,1,1
3,0,0,0,0,0,0,0,0,0,1,1,0
4,0,0,0,0,0,0,1,0,0,1,0,0


<a id="sec3"></a>
## **NLTK STUFF LMAO**

### Stemming

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
#print(stemmed_tokens)